# CA model for Membrane Permeability

Authors:
- H. Malash, 15179192
- S. Bhusal, studentnumber
- L. Pahladsingh, studentnumber
- T. Glansdorp, 12748587

In [1]:
from model import CA_rules, CA_grid, CA_rules_only_water
# import model
import analysis
import numpy as np
import matplotlib.pyplot as plt
import random


This is iteration 1 of the simulation
This is iteration 2 of the simulation
This is iteration 3 of the simulation
This is iteration 4 of the simulation
This is iteration 5 of the simulation
This is iteration 6 of the simulation
This is iteration 7 of the simulation
This is iteration 8 of the simulation
This is iteration 9 of the simulation
This is iteration 10 of the simulation
This is iteration 11 of the simulation
This is iteration 12 of the simulation
This is iteration 13 of the simulation
This is iteration 14 of the simulation
This is iteration 15 of the simulation
This is iteration 16 of the simulation
This is iteration 17 of the simulation
This is iteration 18 of the simulation
This is iteration 19 of the simulation
This is iteration 20 of the simulation
This is iteration 21 of the simulation
This is iteration 22 of the simulation
This is iteration 23 of the simulation
This is iteration 24 of the simulation
This is iteration 25 of the simulation
This is iteration 26 of the simula

KeyboardInterrupt: 

### Counting of unbound water molecules

In [44]:
#for pbw in np.arange(0, 1, 0.1):
pbw = 0.1
final_grid = CA_rules_only_water(CA_grid(), pbw).generate_simulation()
total_height = 55
total_width = 55

unbound_water = analysis.count_unbound_water(final_grid, total_height, total_width)


TypeError: CA_rules_only_water.__init__() takes 2 positional arguments but 3 were given